In [2]:
# --- PREPARACIÓN DE ENTORNO ---
%pip install pandas numpy matplotlib statsmodels openpyxl

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
# (Opcional) tests de raíz unitaria si los usas:
from statsmodels.tsa.stattools import adfuller, kpss

In [5]:
ROOT = Path.cwd().parent     
DATA = ROOT / "data"        
OUTPUT_FIG = ROOT / "output" / "figures"
OUTPUT_TAB = ROOT / "output" / "tables"

OUTPUT_FIG.mkdir(parents=True, exist_ok=True)
OUTPUT_TAB.mkdir(parents=True, exist_ok=True)

In [6]:
excel_path = DATA / "excelbase2.xlsx"
df = pd.read_excel(excel_path, sheet_name="Hoja1", header=1, usecols="B:O")

In [7]:
df.columns = [c.strip().lower() for c in df.columns]
df = df.dropna(how="all")  

In [9]:
fechas = pd.period_range(start="2003-09", end="2019-12", freq="M").to_timestamp()

if len(df) >= len(fechas):
    df = df.iloc[:len(fechas)].copy()
else:
    raise ValueError(f"Faltan {len(fechas)-len(df)} filas para cubrir 2003-09..2019-12. Revisa el Excel.")

df.index = fechas
print(df.head(), "\n", df.tail())
print("Rango temporal:", df.index[0].date(), "→", df.index[-1].date(), "| Obs:", len(df))


                 pbid        ipc           om      tasan  tasar  tasaov  \
2003-09-01  78.347351  58.435976  18703.98083  21.990219   2.75    2.00   
2003-10-01  78.908142  58.464629  19068.25090  21.910154   2.75    2.00   
2003-11-01  75.209304  58.562665  19187.55763  22.284252   2.50    1.75   
2003-12-01  75.752133  58.892929  20112.76777  22.276094   2.50    1.75   
2004-01-01  78.091980  59.209448  20007.22793  23.676494   2.50    1.75   

                  tc      ipi    pmp     pp   ipcu  tasafed     embig  tasaso  
2003-09-01  3.480864  91.3819  138.5  28.31  185.1     1.01  3.547727    0.86  
2003-10-01  3.478205  91.5049  139.3  30.34  184.9     1.01  3.173043    0.83  
2003-11-01  3.477750  92.1265  138.9  31.11  185.0     1.00  3.095500    0.87  
2003-12-01  3.471286  92.1732  139.5  32.13  185.5     0.98  3.179565    0.74  
2004-01-01  3.467167  92.3268  141.4  34.31  186.3     1.00  3.034091    0.75   
                   pbid        ipc           om    tasan  tasar  tas

In [10]:
for col_raw, col_log in [("pbid","lpbi"), ("ipc","lipc"), ("om","lom"), ("tc","ltc")]:
    if col_raw not in df.columns:
        raise KeyError(f"Columna requerida no encontrada: {col_raw}")
    if (df[col_raw] <= 0).any():
        raise ValueError(f"No se puede hacer log: hay valores <= 0 en {col_raw}.")
    df[col_log] = np.log(df[col_raw])

In [11]:
vars_order = ["lpbi", "lipc", "tasar", "tasaso", "lom", "ltc", "embig"]
faltan = [v for v in vars_order if v not in df.columns]
if faltan:
    raise KeyError(f"Faltan columnas: {faltan}")

X = df[vars_order].dropna().copy()

In [12]:
sel = VAR(X).select_order(maxlags=6)
print(sel.summary())
p = 2
print(f"Usaremos p={p}")

 VAR Order Selection (* highlights the minimums) 
      AIC         BIC         FPE         HQIC   
-------------------------------------------------
0      -23.60      -23.48   5.659e-11      -23.55
1      -47.60      -46.64   2.134e-21      -47.21
2      -48.48     -46.69*   8.827e-22     -47.75*
3      -48.52      -45.89   8.495e-22      -47.46
4      -48.55      -45.09  8.322e-22*      -47.15
5     -48.57*      -44.26   8.363e-22      -46.82
6      -48.43      -43.29   9.788e-22      -46.35
-------------------------------------------------
Usaremos p=2


In [13]:
res = VAR(X).fit(p)
print(res.summary())

print("\nEstabilidad (raíces dentro del círculo unitario):")
print(res.is_stable(verbose=True))

steps = 50
alpha_75 = 0.25
irf = res.irf(steps)

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 30, Sep, 2025
Time:                     01:19:39
--------------------------------------------------------------------
No. of Equations:         7.00000    BIC:                   -46.4947
Nobs:                     194.000    HQIC:                  -47.5472
Log likelihood:           2859.63    FPE:                1.09754e-21
AIC:                     -48.2634    Det(Omega_mle):     6.51640e-22
--------------------------------------------------------------------
Results for equation lpbi
               coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------
const             0.108723         0.133281            0.816           0.415
L1.lpbi           0.585864         0.073763            7.943           0.000
L1.lipc           0.347000         0.307283            1.129           0.259


In [14]:
def save_irf(irf_obj, impulse, response, steps, base_outname):
    
    fig = irf_obj.plot(impulse=impulse, response=response, orth=True, signif=alpha_75)
    fig.suptitle(f"shock {impulse} → {response} (75%)", y=1.02)
    fig_path = OUTPUT_FIG / f"{base_outname}.png"
    plt.savefig(fig_path, dpi=300, bbox_inches="tight")
    plt.close()

    
    idx_imp = X.columns.get_loc(impulse)
    idx_resp = X.columns.get_loc(response)
    serie = irf_obj.irfs[:steps+1, idx_resp, idx_imp]
    out = pd.DataFrame({"h": np.arange(steps+1),
                        f"irf_{response}_to_{impulse}": serie})
    csv_path = OUTPUT_TAB / f"{base_outname}.csv"
    out.to_csv(csv_path, index=False)


pares = [
    ("tasaso", "lpbi"),
    ("tasaso", "lipc"),
    ("tasaso", "tasar"),
    ("tasaso", "lom"),
    ("tasaso", "ltc"),
    ("tasaso", "embig"),
    ("tasar",  "lipc"),
]

for imp, resp in pares:
    base_name = f"irf_{resp}_to_{imp}_orth75_step{steps}"
    save_irf(irf, imp, resp, steps, base_name)

print("Listo: IRFs guardados en ../output/figures y ../output/tables")

Listo: IRFs guardados en ../output/figures y ../output/tables
